In [41]:
import time 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

dank_data = pd.read_csv("./data_ocr.csv", delimiter = ",")


In [ ]:
dank_data.shape

In [ ]:
dank_data.insert(4, "words", "Empty") 

In [ ]:
for ind in dank_data.index:    
    if dank_data['words'][ind] == 'Empty':
        title = dank_data['title'][ind]
        text_from_image = dank_data['text_from_image'][ind]
        try:
            if text_from_image == 'Fail':
                text_from_image = ''  
            words = title +  text_from_image
        except:
            words = 'Fail'
        dank_data['words'][ind] = words

In [ ]:
result = 0
for ind in dank_data.index:     

    if (dank_data['words'][ind] == 'Fail'):
        result = result + 1

print(" result = ", result)

In [ ]:
# row in which value of 'text_from_image' column == "Fail"
failed_link_memes = dank_data.apply(lambda x: True if x['words'] == 'Fail' else False , axis=1)
 
# Count number of True in series
numOfRows = len(failed_link_memes[failed_link_memes == True].index)
 
print('Number of Rows in dataframe with broken link : ', numOfRows)

In [ ]:
dank_data.drop(dank_data.index[dank_data['words'] == 'Fail'], inplace = True) 

In [ ]:
# row in which value of 'text_from_image' column == "Fail"
failed_link_memes = dank_data.apply(lambda x: True if x['words'] == 'Fail' else False , axis=1)
 
# Count number of True in series
numOfRows = len(failed_link_memes[failed_link_memes == True].index)
 
print('Number of Rows in dataframe with broken link : ', numOfRows)

In [ ]:
def clean_text(text):
    # remove wrong convertion characters from title
    text = text.map(lambda x: re.sub(r'\\x[00-ff]{2}','',str(x)))
    text = text.str.strip()
    text = text.map(lambda x: re.sub('b"','',str(x)))
    return text

In [ ]:
import re
dank_data['words'] = clean_text(dank_data['words'])

In [ ]:
dank_data['ups_normed'] = dank_data['ups']/dank_data['subscribers']

In [ ]:
# calculate length of the titles
dank_data['TextLength'] = dank_data['words'].str.len()
dank_data.describe()

In [ ]:
dank_data.drop(columns=['text_from_image' ], axis=1, inplace=True)

In [ ]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.models import model_from_json
from keras.models import load_model
import h5py
from keras.models import model_from_json
from keras.models import load_model
from flask import Flask, url_for, request
import json
from nltk.tokenize import RegexpTokenizer

In [ ]:
# load saved LSTM model
from numpy import loadtxt
from keras.models import load_model
 
# load model
LSTM_model = load_model('model.hdf5')
# summarize model.
LSTM_model.summary()

In [ ]:
def live_analyses(trained_model, data, word_idx):
    live_list = []
    live_list_np = np.zeros((56,1))
    # split the sentence into its words and remove any punctuations.
    tokenizer = RegexpTokenizer(r'\w+')
    data_sample_list = tokenizer.tokenize(data)
    labels = np.array(['1','2','3','4','5','6','7','8','9','10'], dtype = "int")
    # get index for the live stage
    data_index = np.array([word_idx[word.lower()] if word.lower() in word_idx else 0 for word in data_sample_list])
    data_index_np = np.array(data_index)
    # padded with zeros of length 56 i.e maximum length
    padded_array = np.zeros(56)
    padded_array[:data_index_np.shape[0]] = data_index_np
    data_index_np_pad = padded_array.astype(int)
    live_list.append(data_index_np_pad)
    live_list_np = np.asarray(live_list)
    # get score from the model
    score = trained_model.predict(live_list_np, batch_size=1, verbose=0)
    single_score = np.round(np.argmax(score)/10, decimals=2) # maximum of the array i.e single band
    # weighted score of top 3 bands
    top_3_index = np.argsort(score)[0][-3:]
    top_3_scores = score[0][top_3_index]
    top_3_weights = top_3_scores/np.sum(top_3_scores)
    single_score_dot = np.round(np.dot(top_3_index, top_3_weights)/10, decimals = 2)
    return single_score_dot, single_score

In [ ]:
word_idx = json.load(open("Data_word_idx.txt"))

In [ ]:
# Source https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925
'''
Loading Gensim and nltk libraries
'''

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
dank_data.insert(5, "processed_words", "Empty") 

In [ ]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [ ]:
import pandas as pd
stemmer = SnowballStemmer("english")
  
for ind in dank_data.index:     
    if dank_data['processed_words'][ind] == 'Empty':
        words = dank_data['words'][ind]
        try:
            processed_words = preprocess(words)
        except:
            processed_words = 'Fail'
        dank_data['processed_words'][ind] = processed_words 

In [ ]:
#dank_data.drop(columns=['Sentiment' ], axis=1, inplace=True)
#dank_data.drop(columns=['Text' ], axis=1, inplace=True)
dank_data['Text'] = 'Empty'
dank_data['Sentiment'] = 0.0
for ind in dank_data.index:
    list_word = dank_data['processed_words'][ind]
    MemeStr = ' '.join(list_word)
    dank_data['Text'][ind] = MemeStr
    MemeStr = MemeStr[:0]

In [ ]:
dank_data.drop(columns=['words'], axis=1, inplace=True)

In [ ]:
#dank_data.drop(columns=['Sentiment' ], axis=1, inplace=True)
#dank_data['Sentiment'] = 0.0
for ind in dank_data.index:
    text = dank_data['Text'][ind][:250]
    result = np.array(live_analyses(LSTM_model,text, word_idx))[0]
    dank_data['Sentiment'][ind] = result

In [ ]:
dank_data.drop(columns=['Text'], axis=1, inplace=True)

In [ ]:
dank_data['word_count'] = 0
for ind in dank_data.index:
  dank_data['word_count'][ind] = len(dank_data['processed_words'][ind])

In [ ]:
dank_data.to_json(r'./dank_data_final.json')
dank_data.to_csv('dank_data_final.csv', index = False)